# 1. Imports
- 데이터 분석과 시각화를 위한 라이브러리 불러오기
- 경고 메시지 무시, 한글 폰트 설정

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings

warnings.filterwarnings('ignore')


plt.rc('font', family='Apple SD Gothic Neo')

from pathlib import Path



In [ ]:
DATA_DIR = Path('data')
if not DATA_DIR.exists():
    DATA_DIR = Path('../Data')

station_df = pd.read_csv(DATA_DIR / 'station_data.csv', index_col=0)
break_df = pd.read_excel(DATA_DIR / '서울시 공공자전거 고장신고 내역_23.1-6.xlsx', engine='openpyxl')
office_df = pd.read_excel(DATA_DIR / '서울특별시 공공자전거 대여소별 이용정보(월별)_23.1-6.xlsx', engine='openpyxl')
history_df = pd.read_csv(DATA_DIR / '서울특별시 공공자전거 대여이력 정보_2301.csv', encoding='cp949')

history_df['반납대여소번호'] = history_df['반납대여소번호'].replace('\\N', np.nan)
history_df['반납대여소번호'] = history_df['반납대여소번호'].astype(float)
office_df['대여건수'] = (office_df['대여건수'].astype(str).str.replace(',', '', regex=False))
office_df['대여건수'] = pd.to_numeric(office_df['대여건수'], errors='coerce').fillna(0).astype(int)
office_df['반납건수'] = (office_df['반납건수'].astype(str).str.replace(',', '', regex=False).replace(' - ', '0'))
office_df['반납건수'] = pd.to_numeric(office_df['반납건수'], errors='coerce').fillna(0).astype(int)

### 타겟 광고를 위한 데이터 분석
#### 상황
- 20대 남성을 타겟으로 한 제품 광고 프로모션 기획
- 어느 대여소가 가장 적합한지 분석


### 성별 및 생년 데이터 정리
- 성별: m → M, f → F, 결측치 → \N
- 생년: 결측치 \N → NaN, float형으로 변환

In [34]:
import datetime

In [35]:
history_df['성별'] = history_df['성별'].replace({'m': 'M', 'f': 'F', np.nan:'\\N'})

In [36]:
history_df['생년'] = history_df['생년'].replace('\\N', np.nan)
history_df['생년'] = history_df['생년'].astype(float)

### 연령대 계산
- 현재 연도를 기준으로 연령 추정
- 연령 구간을 설정해 20-29세 구간 추출

In [37]:
current_year = datetime.datetime.now().year

filtered_history_df = history_df.copy()
filtered_history_df = filtered_history_df[(filtered_history_df['생년'] > 0) & (filtered_history_df['생년'] <= 2010)]

filtered_history_df['연령'] = current_year - filtered_history_df['생년']

age_bins = [13, 20, 30, 40, 50, 60, 70, 80, 90, 100]
age_labels = ['13-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']

filtered_history_df['연령대'] = pd.cut(filtered_history_df['연령'], bins=age_bins, labels=age_labels, right=False)

In [38]:
# 20대 남성이 자주 이용하는 대여소
target_df = filtered_history_df[
    (filtered_history_df['성별'] == 'M') &
    (filtered_history_df['연령대'] == '20-29')
]
target_df.head()

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),생년,성별,이용자종류,대여대여소ID,반납대여소ID,대여시간,반납시간,연령,연령대
5,SPB-35099,2023-01-01 00:01:24,2038,동작역 7번출구,0,2023-01-01 00:05:37,4178.0,이수성결교회 옆,0,4,1005.05,2003.0,M,내국인,ST-696,ST-3065,0,0,22.0,20-29
10,SPB-64954,2023-01-01 00:05:49,2703,서울도시가스 앞,0,2023-01-01 00:06:26,2703.0,서울도시가스 앞,0,1,0.00,2004.0,M,내국인,ST-2018,ST-2018,0,0,21.0,20-29
15,SPB-33542,2023-01-01 00:04:20,1986,태평양물산빌딩,0,2023-01-01 00:07:03,1955.0,디지털입구 교차로,0,2,551.83,1997.0,M,내국인,ST-1521,ST-936,0,0,28.0,20-29
19,SPB-42593,2023-01-01 00:06:59,2703,서울도시가스 앞,0,2023-01-01 00:07:46,2703.0,서울도시가스 앞,0,1,0.00,2004.0,M,내국인,ST-2018,ST-2018,0,0,21.0,20-29
20,SPB-33710,2023-01-01 00:00:34,5061,우장산동 가곡어린이공원앞,0,2023-01-01 00:08:18,2741.0,마곡수명산파크5-6단지,0,7,1207.71,2005.0,M,내국인,ST-2888,ST-2058,0,0,20.0,20-29


In [39]:
target_station_counts = target_df.groupby('대여 대여소명').size().reset_index(name='대여횟수')
target_station_counts.sort_values('대여횟수', ascending=False).head(10)

,대여 대여소명,대여횟수
1942,오목교역 3번출구,576
1040,"발산역 1번, 9번 인근 대여소",513
1771,안암로터리 버스정류장 앞,487
1327,서울과학기술대학교(어학교육원),435
965,목동역5번출구 교통정보센터 앞,414
964,목동역 3번 출구,406
1119,봉림교 교통섬,391
969,목동트라팰리스 웨스턴에비뉴,390
1645,신대방역 2번 출구,386
840,마곡나루역 2번 출구,383


### 전체 대여 대비 20대 남성 비율
- 대여소별 전체 대여 횟수와 20대 남성 대여 횟수 병합
- 20대 남성 비율 계산 후 상위 대여소 확인

In [40]:
total_rentals_per_station = filtered_history_df.groupby('대여 대여소명').size().reset_index(name='전체대여횟수')
total_rentals_per_station.head()

,대여 대여소명,전체대여횟수
0,(시립)고덕평생학습관,273
1,(재)FITI시험연구원,1026
2,3호선 매봉역 3번출구앞,710
3,419민주묘지 입구사거리,351
4,419민주묘지역,243


In [41]:
merged_df = total_rentals_per_station.merge(target_station_counts, on='대여 대여소명', how='left')
merged_df.head()

,대여 대여소명,전체대여횟수,대여횟수
0,(시립)고덕평생학습관,273,16.0
1,(재)FITI시험연구원,1026,22.0
2,3호선 매봉역 3번출구앞,710,62.0
3,419민주묘지 입구사거리,351,26.0
4,419민주묘지역,243,26.0


In [42]:
merged_df['20대남성비율'] = merged_df['대여횟수']/merged_df['전체대여횟수']
merged_df.sort_values('20대남성비율', ascending=False).head(10)

,대여 대여소명,전체대여횟수,대여횟수,20대남성비율
371,구룡역 4번 출구,41,22.0,0.536585
1546,세종대학교(학술정보원),114,57.0,0.500000
727,동선아파트 앞,116,56.0,0.482759
1022,문화비축기지,67,31.0,0.462687
1543,세종대학교,84,38.0,0.452381
1340,서울과학기술대학교(어학교육원),977,435.0,0.445241
984,목양전원교회 앞 로터리,99,42.0,0.424242
1394,서울시립대 앞,85,36.0,0.423529
1218,삼육대 도서관,68,28.0,0.411765
444,금천구립 독산도서관,217,81.0,0.373272


In [43]:
# 20대 남성 대여 비율
total_rentals = len(filtered_history_df)
target_rentals = len(target_df)

target_rentals / total_rentals

0.10366260766888405

### KMeans를 활용한 비슷한 성향의 대여소 찾기
#### 상황
- 이전 광고에서 좋은 성과가 나왔다.
- 특히 '서울과기대' 에서의 광고에서 높은 효율로 성과가 나왔다.
- 그래서 이번에는 다른 곳에 더 확장을 하기로 했다.
#### 방법
- Clustering을 활용해서, 서울과기대와 비슷한 성향의 대여소를 찾아서 광고한다.

#### 시간대, 연령대, 성별별 대여소의 이용량

In [15]:
from sklearn.cluster import KMeans

In [44]:
filtered_history_df['대여시간'] = pd.to_datetime(filtered_history_df['대여일시']).dt.hour

In [45]:
# 시간대별
hourly_groupby_df = filtered_history_df.groupby(['대여 대여소명', '대여시간']).size().unstack(fill_value=0)
hourly_groupby_df.head()

대여시간,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
대여 대여소명,,,,,,,,,,,,,,,,,,,,,
(시립)고덕평생학습관,1,2,2,4,1,0,0,14,3,11,...,16,12,21,12,28,14,22,21,7,1
(재)FITI시험연구원,13,8,1,2,0,1,9,85,195,71,...,22,37,96,53,143,37,39,20,10,16
3호선 매봉역 3번출구앞,18,3,3,3,0,5,18,13,104,57,...,43,28,36,48,88,45,23,35,32,17
419민주묘지 입구사거리,16,10,4,7,5,6,12,27,5,41,...,19,13,24,11,19,10,13,21,8,6
419민주묘지역,7,9,1,1,0,1,2,4,9,13,...,15,13,12,16,19,16,8,9,8,11


In [46]:
# 연령대
age_groupby_df = filtered_history_df.groupby(['대여 대여소명', '연령대']).size().unstack(fill_value=0)
age_groupby_df.head()

연령대,13-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
대여 대여소명,,,,,,,,,
(시립)고덕평생학습관,12,53,72,54,53,27,1,0,0
(재)FITI시험연구원,0,126,695,106,70,27,0,0,0
3호선 매봉역 3번출구앞,3,130,255,125,115,68,11,2,0
419민주묘지 입구사거리,5,47,98,22,56,123,0,0,0
419민주묘지역,3,57,73,20,38,44,8,0,0


In [47]:
# 성별
gender_groupby_df = filtered_history_df.groupby(['대여 대여소명', '성별']).size().unstack(fill_value=0)
gender_groupby_df.head()

성별,F,M,\N
대여 대여소명,,,
(시립)고덕평생학습관,78,122,73
(재)FITI시험연구원,239,423,364
3호선 매봉역 3번출구앞,136,373,201
419민주묘지 입구사거리,66,209,76
419민주묘지역,50,145,48


### 종합 피처 데이터프레임 구성
- 대여소 × (성별, 연령대, 시간대) 조합별 count pivot
- 여러 특성 데이터프레임을 하나로 합쳐 클러스터링 입력값으로 사용

In [48]:
general_groupby_df = filtered_history_df.groupby(['대여 대여소명', '성별', '연령대', '대여시간']).size()
general_groupby_df = general_groupby_df.unstack("성별")
general_groupby_df = general_groupby_df.unstack("연령대")
general_groupby_df = general_groupby_df.unstack("대여시간")
general_groupby_df.head()

성별                F                             ...  \N                       \
연령대           13-19                             ... 90+                        
대여시간             0  1  2  3  4  5  6  7  8  9   ...  14 15 16 17 18 19 20 21   
대여 대여소명                                         ...                            
(시립)고덕평생학습관       0  0  0  0  0  0  0  0  0  0  ...   0  0  0  0  0  0  0  0   
(재)FITI시험연구원      0  0  0  0  0  0  0  0  0  0  ...   0  0  0  0  0  0  0  0   
3호선 매봉역 3번출구앞     0  0  0  0  0  0  0  0  0  0  ...   0  0  0  0  0  0  0  0   
419민주묘지 입구사거리     0  0  0  0  0  0  0  0  0  0  ...   0  0  0  0  0  0  0  0   
419민주묘지역          0  0  0  0  0  0  0  0  0  0  ...   0  0  0  0  0  0  0  0   

성별                   
연령대                  
대여시간          22 23  
대여 대여소명              
(시립)고덕평생학습관    0  0  
(재)FITI시험연구원   0  0  
3호선 매봉역 3번출구앞  0  0  
419민주묘지 입구사거리  0  0  
419민주묘지역       0  0  

[5 rows x 648 columns]

In [49]:
general_groupby_df.columns = [" ".join(
    [str(c) for c in column]
) for column in general_groupby_df.columns]

In [51]:
general_groupby_df

,F 13-19 0,F 13-19 1,F 13-19 2,F 13-19 3,F 13-19 4,F 13-19 5,F 13-19 6,F 13-19 7,F 13-19 8,F 13-19 9,...,\N 90+ 14,\N 90+ 15,\N 90+ 16,\N 90+ 17,\N 90+ 18,\N 90+ 19,\N 90+ 20,\N 90+ 21,\N 90+ 22,\N 90+ 23
대여 대여소명,,,,,,,,,,,,,,,,,,,,,
(시립)고덕평생학습관,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(재)FITI시험연구원,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3호선 매봉역 3번출구앞,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
419민주묘지 입구사거리,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
419민주묘지역,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
흥국사 정류장,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
희성오피앙,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
희훈타워빌 앞,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
features_df = pd.concat([general_groupby_df, gender_groupby_df, age_groupby_df, hourly_groupby_df], axis=1)
features_df

,F 13-19 0,F 13-19 1,F 13-19 2,F 13-19 3,F 13-19 4,F 13-19 5,F 13-19 6,F 13-19 7,F 13-19 8,F 13-19 9,...,14,15,16,17,18,19,20,21,22,23
대여 대여소명,,,,,,,,,,,,,,,,,,,,,
(시립)고덕평생학습관,0,0,0,0,0,0,0,0,0,0,...,16,12,21,12,28,14,22,21,7,1
(재)FITI시험연구원,0,0,0,0,0,0,0,0,0,0,...,22,37,96,53,143,37,39,20,10,16
3호선 매봉역 3번출구앞,0,0,0,0,0,0,0,0,0,0,...,43,28,36,48,88,45,23,35,32,17
419민주묘지 입구사거리,0,0,0,0,0,0,0,0,0,0,...,19,13,24,11,19,10,13,21,8,6
419민주묘지역,0,0,0,0,0,0,0,0,0,0,...,15,13,12,16,19,16,8,9,8,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
흥국사 정류장,0,0,0,0,0,0,0,0,0,0,...,0,3,3,2,0,1,0,0,0,1
희성오피앙,0,0,0,0,0,0,0,0,0,0,...,32,44,30,30,49,34,25,33,9,8
희훈타워빌 앞,0,0,0,0,0,0,0,0,0,0,...,54,56,34,69,112,45,31,37,29,12


### KMeans 군집화 수행
- 30개의 클러스터로 대여소를 분류

In [53]:
from sklearn.cluster import KMeans

In [54]:
features_df.columns = features_df.columns.astype(str)
kmeans = KMeans(n_clusters=30, random_state=13)
features_df['cluster'] = kmeans.fit_predict(features_df)
features_df

,F 13-19 0,F 13-19 1,F 13-19 2,F 13-19 3,F 13-19 4,F 13-19 5,F 13-19 6,F 13-19 7,F 13-19 8,F 13-19 9,...,15,16,17,18,19,20,21,22,23,cluster
대여 대여소명,,,,,,,,,,,,,,,,,,,,,
(시립)고덕평생학습관,0,0,0,0,0,0,0,0,0,0,...,12,21,12,28,14,22,21,7,1,17
(재)FITI시험연구원,0,0,0,0,0,0,0,0,0,0,...,37,96,53,143,37,39,20,10,16,19
3호선 매봉역 3번출구앞,0,0,0,0,0,0,0,0,0,0,...,28,36,48,88,45,23,35,32,17,6
419민주묘지 입구사거리,0,0,0,0,0,0,0,0,0,0,...,13,24,11,19,10,13,21,8,6,1
419민주묘지역,0,0,0,0,0,0,0,0,0,0,...,13,12,16,19,16,8,9,8,11,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
흥국사 정류장,0,0,0,0,0,0,0,0,0,0,...,3,3,2,0,1,0,0,0,1,5
희성오피앙,0,0,0,0,0,0,0,0,0,0,...,44,30,30,49,34,25,33,9,8,27
희훈타워빌 앞,0,0,0,0,0,0,0,0,0,0,...,56,34,69,112,45,31,37,29,12,19


### 서울과기대와 같은 클러스터에 속한 대여소 찾기

In [55]:
# 과기대와 비슷한 성향인 대여소 찾기
cluster = features_df.loc['서울과학기술대학교(어학교육원)']['cluster']
features_df[features_df['cluster'] == cluster].index

Index(['강북구청사거리(수유역 8번출구)', '건국대학교 학생회관', '공릉역 1번 출구 앞', '과기대 입구(우)',
       '노원역7번출구', '도깨비시장', '동신아파트 후문 옆', '동양미래대학교 정문 옆', '동해문화예술관앞', '둔촌동역교차로',
       '둔촌사거리', '등나무 근린공원(시립북서울미술관 앞)', '맥도날드 서울둔촌DT점 앞', '목동사거리 부근',
       '상계주공1단지 버스정류장 옆(대원빌딩 앞)', '서울과학기술대학교(어학교육원)', '서울시립대 정문 앞', '성균관대 정문',
       '신대방삼거리역 6번출구쪽', '양평동6차현대아파트 앞', '어린이대공원역 3번출구 앞', '연신내역 5번출구',
       '온곡초교 교차로', '외국어대 정문 앞', '우장산동 가곡어린이공원앞', '원일교회', '제기역1번출구',
       '중앙하이츠 아파트 입구', '홍대입구역 8번출구 앞', '홍익병원앞 교차로', '화양동 우체국'],
      dtype='object', name='대여 대여소명')

In [56]:
len(features_df[features_df['cluster'] == cluster].index)

31